### Machine Hack Data Scientist Salary Prediction Problem.



### Import Libraries

In [193]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

#### Import Data

In [32]:
train_df = pd.read_csv('Final_Train_Dataset.csv', index_col=0)

test_df = pd.read_csv('Final_Test_Dataset.csv')

sample_df = pd.read_excel('sample_submission.xlsx')

In [33]:
train_df.head()

,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
0,5-7 yrs,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,NaN,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,10-17 yrs,He should have handled a team of atleast 5-6 d...,Head SCM,NaN,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458
2,5-9 yrs,Must be an effective communicator (written & s...,Deputy Manager - Talent Management & Leadershi...,Analytics,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195
3,7-10 yrs,7 - 10 years of overall experience in data e...,Associate Manager Data Engineering,Analytics,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313
4,1-3 yrs,Chartered Accountancy degree or MBA in Finance...,TS- GSA- Senior Analyst,NaN,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305


In [34]:
test_df.head()

,experience,job_description,job_desig,job_type,key_skills,location,company_name_encoded
0,7-12 yrs,Professional experience in Java/J2EE based ser...,IT Technology Senior Consultant/java/ J2ee/ Se...,NaN,"Java, J2Ee, Tomcat, JBoss, Weblogic, Oracle, E...",Bengaluru,2066
1,0-5 yrs,We are looking for 20+ Fresher/Experienced Can...,Medical Billing Process | International KPO | ...,NaN,"Medical Billing, Insurance Processing",Ahmedabad(Sola),2629
2,3-6 yrs,Should understand overall integration framewor...,Oracle Sales Cloud Functional Consultant,NaN,"Oracle Sales, Functional Consultancy, Troubles...",Bengaluru,2448
3,0-3 yrs,NaN,Looking For Freshers WHO WANT To Work WITH US,NaN,"offline, online, internet, part time, home bas...","Delhi NCR, Chennai, Hyderabad, Gurgaon, Luckno...",2711
4,0-5 yrs,NaN,Process Associate / Sr Process Associate / Tec...,NaN,"voice support, analytical skills, Process asso...",Hyderabad,40


In [31]:
sample_df.head()

,salary
0,6to10
1,10to15
2,15to25
3,10to15
4,3to6


In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19802 entries, 0 to 19801
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   experience            19802 non-null  object
 1   job_description       15384 non-null  object
 2   job_desig             19802 non-null  object
 3   job_type              4797 non-null   object
 4   key_skills            19801 non-null  object
 5   location              19802 non-null  object
 6   salary                19802 non-null  object
 7   company_name_encoded  19802 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 1.4+ MB


In [35]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6601 entries, 0 to 6600
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   experience            6601 non-null   object
 1   job_description       5079 non-null   object
 2   job_desig             6601 non-null   object
 3   job_type              1637 non-null   object
 4   key_skills            6601 non-null   object
 5   location              6601 non-null   object
 6   company_name_encoded  6601 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 361.1+ KB


In [12]:
train_df['experience'].value_counts()

5-10 yrs     1274
2-5 yrs      1188
3-8 yrs       922
2-7 yrs       832
4-9 yrs       678
             ... 
17-24 yrs       1
16-20 yrs       1
20-25 yrs       1
11-12 yrs       1
8-8 yrs         1
Name: experience, Length: 129, dtype: int64

In [14]:
train_df['job_description'].nunique()

9059

In [18]:
train_df[train_df['job_description'].isna()]

,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
26,3-7 yrs,NaN,"E Commerce Manager,",NaN,"amazon, E - commerce, PORTAL HANDLING, Busines...",Ahmedabad(Bopal),3to6,895
37,2-5 yrs,NaN,IBM Netezza,Analytics,"SQL, Oracle, Python, Analytics, Data migration...",Bengaluru,6to10,190
41,2-5 yrs,NaN,Business Development Executive,NaN,"cold calling, digital marketing, social media,...",Chennai,3to6,4946
44,5-9 yrs,NaN,CN - Strategy - MC - Resources - Utilities - 06,Analytics,"Analytics, System integration, Asset managemen...",Gurgaon,10to15,191
58,0-5 yrs,NaN,Data Entry/home Base Job/ Online Work/part Tim...,NaN,"part time, freelancing, data entry, present jo...","Udaipur, Ajmer, Kota, Sangrur, Aurangabad, Muz...",0to3,1153
...,...,...,...,...,...,...,...,...
19775,1-5 yrs,NaN,R1rcm- Accretive Analyst/ Senior Analyst Backe...,NaN,"bpo, kpo, non bpo, night shift, us process, uk...",Gurgaon,0to3,2094
19776,0-1 yrs,NaN,Software Sales CRM,analytics,"software sales, selling, predictive analytics,...",Hyderabad(Madhapur),0to3,269
19782,1-4 yrs,NaN,"Credit Manager,",NaN,"Financial Analysis, Credit Analysis, Finance, ...",Delhi NCR(Peeragarhi),0to3,3940
19797,12-18 yrs,NaN,"Director, Medical Coding Training",NaN,"Medical Coding, ICD - 10, US Healthcare, RCM, ...",Hyderabad,15to25,2245


In [20]:
train_df['job_type'].unique()

array([nan, 'Analytics', 'analytics', 'Analytic', 'ANALYTICS', 'analytic'],
      dtype=object)

In [21]:
train_df['key_skills'].nunique()

12951

In [23]:
train_df['location'].nunique()

1504

In [27]:
train_df['location'].value_counts(ascending = True).head(100)

Delhi NCR(Sector-18 Noida)                                1
Pune, Thane                                               1
Delhi NCR, Bengaluru, Chennai                             1
Delhi NCR, Greater Noida(Greater Noida West)              1
Hyderabad, Chandigarh, Panchkula                          1
                                                         ..
Bengaluru(Chikka Bommasandra)                             1
Pune, Mumbai, Delhi NCR                                   1
Mumbai(Churchgate)                                        1
Chennai(Adyar), Coimbatore                                1
Delhi NCR, Mumbai, Bengaluru, Ahmedabad, Kolkata, Pune    1
Name: location, Length: 100, dtype: int64

In [83]:
train_drop = train_df.drop(['job_description', 'job_type'], axis=1)

In [38]:
test_drop = test_df.drop(['job_description', 'job_type'], axis=1)

In [42]:
sal = train_drop['salary'].astype('category')

In [52]:
sal_dict = {}
j=0
for i in sal.cat.categories:
    sal_dict[j] = i
    j+=1
sal_dict

{0: '0to3', 1: '10to15', 2: '15to25', 3: '25to50', 4: '3to6', 5: '6to10'}

#### Label Encoder

In [53]:
train_drop.head()

,experience,job_desig,key_skills,location,salary,company_name_encoded
0,5-7 yrs,Senior Exploit and Vulnerability Researcher,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,10-17 yrs,Head SCM,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458
2,5-9 yrs,Deputy Manager - Talent Management & Leadershi...,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195
3,7-10 yrs,Associate Manager Data Engineering,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313
4,1-3 yrs,TS- GSA- Senior Analyst,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305


In [57]:
train_drop[['experience', 'job_desig', 'key_skills', 'location']].head()

,experience,job_desig,key_skills,location
0,5-7 yrs,Senior Exploit and Vulnerability Researcher,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri)
1,10-17 yrs,Head SCM,"ppc, logistics, inventory management, supply c...",Sonepat
2,5-9 yrs,Deputy Manager - Talent Management & Leadershi...,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR
3,7-10 yrs,Associate Manager Data Engineering,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru
4,1-3 yrs,TS- GSA- Senior Analyst,"accounting, finance, cash flow, financial plan...",Gurgaon


In [59]:
cat_col = ['experience', 'job_desig', 'key_skills', 'location']

In [84]:
df = pd.concat([train_drop, test_drop])

In [85]:
df.dropna(subset=['key_skills'], inplace=True)

In [86]:
df.isna().sum()

experience                 0
job_desig                  0
key_skills                 0
location                   0
salary                  6601
company_name_encoded       0
dtype: int64

In [87]:
df['experience']

0         5-7 yrs
1       10-17 yrs
2         5-9 yrs
3        7-10 yrs
4         1-3 yrs
          ...    
6596      4-7 yrs
6597      1-5 yrs
6598     5-10 yrs
6599     7-12 yrs
6600      3-8 yrs
Name: experience, Length: 26402, dtype: object

In [88]:
le = LabelEncoder()
def label(x):
    le.fit(df[x])
    df[x] = le.transform(df[x])

In [89]:
for i in cat_col:
    print(i)
    label(i)
    

experience
job_desig
key_skills
location


### spliting

In [98]:
df[df['salary'].isnull()].drop('salary',axis=1)

,experience,job_desig,key_skills,location,company_name_encoded
0,118,5761,4904,130,2066
1,5,7703,5938,30,2629
2,96,8435,6304,130,2448
3,3,6703,13371,709,2711
4,5,8777,15314,984,40
...,...,...,...,...,...
6596,102,1872,9701,1177,2692
6597,10,9794,1775,924,104
6598,105,839,10266,1177,2025
6599,118,1320,1098,984,2512


In [110]:
train_split = df[df['salary'].notnull()]

test_split = df[df['salary'].isnull()].drop('salary', axis=1)

In [111]:
train_split['salary']=train_split.salary.astype('category').cat.codes

<ipython-input-111-5f1636dce35b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_split['salary']=train_split.salary.astype('category').cat.codes


#### Train Test split

In [132]:
X=train_split.drop('salary',axis=1)

y=train_split['salary']

In [113]:
X_train, X_test, y_train, y_test = train_test_split(train_split.drop('salary',axis=1), train_split['salary'], test_size=0.3, random_state=123)

In [115]:
test_split.head()

,experience,job_desig,key_skills,location,company_name_encoded
0,118,5761,4904,130,2066
1,5,7703,5938,30,2629
2,96,8435,6304,130,2448
3,3,6703,13371,709,2711
4,5,8777,15314,984,40


#### Model

In [152]:
def metrics (y_true, y_pred):
    print(confusion_matrix(y_true, y_pred),'\n\n')
    print('Accuracy Score:', accuracy_score(y_true, y_pred),'\n\n')
    print(classification_report(y_true, y_pred),'\n\n')
    
def predictions(model, X_train=X_train,  X_test=X_test,  y_train=y_train,  y_test=y_test, full=False):
    model.fit(X_train, y_train)
    
    train_pred = model.predict(X_train)
    
    test_pred = model.predict(X_test)
    
    actual = [y_train, y_test]
    
    pred = [train_pred, test_pred]
    
    data = ['Train', 'Test']
    
    for i in range(2):
        print(data[i]+' Metrics\n')
        metrics(actual[i], pred[i])
        
    if full:
        full_predict(model)

In [153]:
def full_predict(model, X=X, y=y, test=test_split, sal_dict=sal_dict):
    model.fit(X,y)
    pred = model.predict(test)
    sample_df['salary'] = pred
    sample_df['salary'] = sample_df['salary'].map(sal_dict)
    sample_df.to_csv(str(model)[:10]+'pred.csv', index=False)
    print('DONE')

#### Logistic Regression

In [154]:
log = LogisticRegression()

predictions(log,full=True)

Train Metrics

[[1539  553  121    0   28   32]
 [ 396 2567  158    0   32   29]
 [ 497 2180  128    0   38   28]
 [ 384  551  104    0   22   11]
 [ 684 1055  155    0   25   80]
 [ 355 1862  165    0   27   54]] 


Accuracy Score: 0.3111832611832612 


              precision    recall  f1-score   support

           0       0.40      0.68      0.50      2273
           1       0.29      0.81      0.43      3182
           2       0.15      0.04      0.07      2871
           3       0.00      0.00      0.00      1072
           4       0.15      0.01      0.02      1999
           5       0.23      0.02      0.04      2463

    accuracy                           0.31     13860
   macro avg       0.20      0.26      0.18     13860
weighted avg       0.23      0.31      0.21     13860
 


Test Metrics

[[ 640  230   61    0   18   23]
 [ 165 1064   73    0    3   13]
 [ 206  970   53    0   12   13]
 [ 170  282   40    0   10    1]
 [ 291  437   47    0   15   35]
 [ 162  820   59    

### KNN

In [157]:
knn = KNeighborsClassifier()

predictions(knn, full=True)

Train Metrics

[[1493  249  184   44  170  133]
 [ 401 1966  390   57  159  209]
 [ 332  724 1458   99  115  143]
 [ 143  196  265  380   36   52]
 [ 459  341  201   39  774  185]
 [ 352  647  344   64  268  788]] 


Accuracy Score: 0.49487734487734486 


              precision    recall  f1-score   support

           0       0.47      0.66      0.55      2273
           1       0.48      0.62      0.54      3182
           2       0.51      0.51      0.51      2871
           3       0.56      0.35      0.43      1072
           4       0.51      0.39      0.44      1999
           5       0.52      0.32      0.40      2463

    accuracy                           0.49     13860
   macro avg       0.51      0.47      0.48     13860
weighted avg       0.50      0.49      0.49     13860
 


Test Metrics

[[392 186 142  27 121 104]
 [237 462 310  49  98 162]
 [214 459 338  56  88  99]
 [ 80 119 138 113  21  32]
 [219 188 101  25 173 119]
 [196 378 194  26 135 140]] 


Accuracy Score: 0.

#### Naive Bayes

In [160]:
nb = MultinomialNB()

predictions(nb, full=True)

Train Metrics

[[ 617    4  100  743  579  230]
 [ 436   30  331 1100  823  462]
 [ 347   26  357 1051  707  383]
 [ 152    6  130  427  214  143]
 [ 344   12  147  555  649  292]
 [ 328   11  209  750  717  448]] 


Accuracy Score: 0.1823953823953824 


              precision    recall  f1-score   support

           0       0.28      0.27      0.27      2273
           1       0.34      0.01      0.02      3182
           2       0.28      0.12      0.17      2871
           3       0.09      0.40      0.15      1072
           4       0.18      0.32      0.23      1999
           5       0.23      0.18      0.20      2463

    accuracy                           0.18     13860
   macro avg       0.23      0.22      0.17     13860
weighted avg       0.25      0.18      0.17     13860
 


Test Metrics

[[269   1  39 310 259  94]
 [180   9 131 475 343 180]
 [175   6 143 443 311 176]
 [ 84   3  65 179 104  68]
 [154   9  49 236 274 103]
 [137  13 104 333 309 173]] 


Accuracy Score: 0.1

#### Decision Tree

In [161]:
dtree = DecisionTreeClassifier()

predictions(dtree, full=True)

Train Metrics

[[2248    0    0    0   25    0]
 [   0 3133   46    0    3    0]
 [   0  539 2303   29    0    0]
 [   0    2  248  822    0    0]
 [ 267   31   17    0 1684    0]
 [  53  301  167    8  287 1647]] 


Accuracy Score: 0.854040404040404 


              precision    recall  f1-score   support

           0       0.88      0.99      0.93      2273
           1       0.78      0.98      0.87      3182
           2       0.83      0.80      0.81      2871
           3       0.96      0.77      0.85      1072
           4       0.84      0.84      0.84      1999
           5       1.00      0.67      0.80      2463

    accuracy                           0.85     13860
   macro avg       0.88      0.84      0.85     13860
weighted avg       0.87      0.85      0.85     13860
 


Test Metrics

[[512  74  39   3 226 118]
 [ 58 425 425  45  85 280]
 [ 26 466 380 175  51 156]
 [  8  46 216 208   5  20]
 [232  94  50   3 236 210]
 [114 329 167  13 244 202]] 


Accuracy Score: 0.33

#### Bagging 

In [163]:
bag = BaggingClassifier()

predictions(bag, full=True)

Train Metrics

[[2086    2    0    0  160   25]
 [   9 2699  320    2   18  134]
 [   5  288 2368  138    8   64]
 [   1    4  149  911    0    7]
 [ 153   25   15    0 1668  138]
 [  43  190  115    5  164 1946]] 


Accuracy Score: 0.8425685425685425 


              precision    recall  f1-score   support

           0       0.91      0.92      0.91      2273
           1       0.84      0.85      0.84      3182
           2       0.80      0.82      0.81      2871
           3       0.86      0.85      0.86      1072
           4       0.83      0.83      0.83      1999
           5       0.84      0.79      0.81      2463

    accuracy                           0.84     13860
   macro avg       0.85      0.84      0.85     13860
weighted avg       0.84      0.84      0.84     13860
 


Test Metrics

[[564  64  28   0 219  97]
 [ 52 434 455  29  66 282]
 [ 21 473 427 159  27 147]
 [  4  37 222 223   3  14]
 [229 104  41   0 226 225]
 [ 96 353 165  10 234 211]] 


Accuracy Score: 0.3

#### Random Forest

In [164]:
rf = RandomForestClassifier()

predictions(rf, full=True)

Train Metrics

[[2122    0    0    0  125   26]
 [   0 2720  317    1   13  131]
 [   0  264 2413  116    9   69]
 [   0    0  163  906    0    3]
 [ 167   21    8    0 1664  139]
 [  27  167  103    5  149 2012]] 


Accuracy Score: 0.854040404040404 


              precision    recall  f1-score   support

           0       0.92      0.93      0.92      2273
           1       0.86      0.85      0.86      3182
           2       0.80      0.84      0.82      2871
           3       0.88      0.85      0.86      1072
           4       0.85      0.83      0.84      1999
           5       0.85      0.82      0.83      2463

    accuracy                           0.85     13860
   macro avg       0.86      0.85      0.86     13860
weighted avg       0.85      0.85      0.85     13860
 


Test Metrics

[[610  61  20   1 194  86]
 [ 51 438 475  12  51 291]
 [ 12 481 439 149  27 146]
 [  3  36 228 222   1  13]
 [245  98  37   0 217 228]
 [101 391 151  10 212 204]] 


Accuracy Score: 0.35

#### ADA Boost

In [166]:
ada = AdaBoostClassifier()

predictions(ada, full=True)

Train Metrics

[[1652  162   26    2  140  291]
 [ 225 1239 1025   84   90  519]
 [  86  892 1360  248   45  240]
 [  14  110  669  241    1   37]
 [ 691  398   50    0  211  649]
 [ 319  781  330   17  162  854]] 


Accuracy Score: 0.4009379509379509 


              precision    recall  f1-score   support

           0       0.55      0.73      0.63      2273
           1       0.35      0.39      0.37      3182
           2       0.39      0.47      0.43      2871
           3       0.41      0.22      0.29      1072
           4       0.33      0.11      0.16      1999
           5       0.33      0.35      0.34      2463

    accuracy                           0.40     13860
   macro avg       0.39      0.38      0.37     13860
weighted avg       0.39      0.40      0.38     13860
 


Test Metrics

[[697  70  22   2  49 132]
 [ 76 485 430  35  37 255]
 [ 38 412 587  90  19 108]
 [  3  53 320 102   4  21]
 [278 163  32   0  79 273]
 [150 357 151   8  53 350]] 


Accuracy Score: 0.3

#### Gradient Boosting

In [167]:
grad = GradientBoostingClassifier()

predictions(grad,full = True)

Train Metrics

[[1755  158   42    5  163  150]
 [ 193 1682  856   67  118  266]
 [  78  819 1592  196   57  129]
 [  10  113  432  487    7   23]
 [ 625  374   95    2  538  365]
 [ 269  761  365   22  248  798]] 


Accuracy Score: 0.49437229437229435 


              precision    recall  f1-score   support

           0       0.60      0.77      0.67      2273
           1       0.43      0.53      0.47      3182
           2       0.47      0.55      0.51      2871
           3       0.63      0.45      0.53      1072
           4       0.48      0.27      0.34      1999
           5       0.46      0.32      0.38      2463

    accuracy                           0.49     13860
   macro avg       0.51      0.48      0.48     13860
weighted avg       0.49      0.49      0.48     13860
 


Test Metrics

[[706  71  31   2  89  73]
 [ 82 582 412  26  47 169]
 [ 28 443 593 104  28  58]
 [  4  56 240 182   9  12]
 [286 162  45   2 156 174]
 [125 388 151  12 128 265]] 


Accuracy Score: 0.

#### Light GBM

In [169]:
li = LGBMClassifier()

predictions(li, full=True)

Train Metrics

[[1977   71   42    2  119   62]
 [  95 2104  654   31   77  221]
 [  39  491 2065  137   37  102]
 [   2   75  276  701    4   14]
 [ 459  157   71    1 1064  247]
 [ 163  511  334   16  199 1240]] 


Accuracy Score: 0.6602453102453102 


              precision    recall  f1-score   support

           0       0.72      0.87      0.79      2273
           1       0.62      0.66      0.64      3182
           2       0.60      0.72      0.65      2871
           3       0.79      0.65      0.72      1072
           4       0.71      0.53      0.61      1999
           5       0.66      0.50      0.57      2463

    accuracy                           0.66     13860
   macro avg       0.68      0.66      0.66     13860
weighted avg       0.66      0.66      0.66     13860
 


Test Metrics

[[684  46  29   2 143  68]
 [ 57 537 437  19  64 204]
 [ 19 405 612 112  30  76]
 [  5  48 218 221   4   7]
 [263 107  51   0 208 196]
 [ 93 360 169   9 177 261]] 


Accuracy Score: 0.4

#### XG Boost

In [171]:
xgb = XGBClassifier()

predictions(xgb, full=True)

[10:41:36] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train Metrics

[[2016   63   24    0  119   51]
 [  59 2307  533   24   56  203]
 [  24  394 2198  117   35  103]
 [   3   73  230  750    4   12]
 [ 350  139   64    0 1208  238]
 [ 124  418  241   16  180 1484]] 


Accuracy Score: 0.7188311688311688 


              precision    recall  f1-score   support

           0       0.78      0.89      0.83      2273
           1       0.68      0.73      0.70      3182
           2       0.67      0.77      0.71      2871
           3       0.83      0.70      0.76      1072
           4       0.75      0.60      0.67      1999
           5       0.71      0.60      0.65      2463

    accuracy                           0.72     13860
   macro avg       0.74      0.71      0.72     13860
weig

In [177]:
params = {
    'learning_rate' : [0.05, 0.15, 0.20, 0.25],
    'max_depth' : [3, 4, 5, 6],
    'min_child_weight': [1, 3, 5],
    'gamma' : [0.1,0.2,0.3], #less than 1 make sure
    'colsample_bytree':[0.3,0.4,0.5] #less than 1
}

grid = GridSearchCV(xgb, params, verbose=3, n_jobs=-1, cv=2)

grid.fit(X_train, y_train)



Fitting 2 folds for each of 432 candidates, totalling 864 fits
[11:26:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_estimators=100, n_jobs=4,
                                     num_parallel_tree=1,
                                     objective='multi:softprob', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=None, subsample=1,
                      

In [186]:
list(grid.best_params_.keys())

['colsample_bytree', 'gamma', 'learning_rate', 'max_depth', 'min_child_weight']

In [185]:
list(grid.best_params_.values())

[0.4, 0.1, 0.05, 6, 1]

In [189]:
best_xgb=XGBClassifier(colsample_bytree=0.4, gamma=0.1, learning_rate=0.05, max_depth=6, min_child_weight=1)

In [192]:
predictions(best_xgb,full=True)

[12:42:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train Metrics

[[1849  146   63    3  107  105]
 [ 185 1859  864   26   70  178]
 [  80  627 1933   91   35  105]
 [  14  111  451  472    4   20]
 [ 636  334  132    0  586  311]
 [ 281  728  387   15  171  881]] 


Accuracy Score: 0.5468975468975469 


              precision    recall  f1-score   support

           0       0.61      0.81      0.70      2273
           1       0.49      0.58      0.53      3182
           2       0.50      0.67      0.58      2871
           3       0.78      0.44      0.56      1072
           4       0.60      0.29      0.39      1999
           5       0.55      0.36      0.43      2463

    accuracy                           0.55     13860
   macro avg       0.59      0.53      0.53     13860
weig

### CAT Boost

In [174]:
cat = CatBoostClassifier()

predictions(cat, full=True)

Learning rate set to 0.09048
0:	learn: 1.7376596	total: 25.4ms	remaining: 25.3s
1:	learn: 1.6930916	total: 45ms	remaining: 22.5s
2:	learn: 1.6535020	total: 74.3ms	remaining: 24.7s
3:	learn: 1.6188971	total: 96.8ms	remaining: 24.1s
4:	learn: 1.5925568	total: 118ms	remaining: 23.4s
5:	learn: 1.5662176	total: 137ms	remaining: 22.8s
6:	learn: 1.5476141	total: 166ms	remaining: 23.5s
7:	learn: 1.5285230	total: 209ms	remaining: 25.9s
8:	learn: 1.5109626	total: 236ms	remaining: 26s
9:	learn: 1.4975107	total: 255ms	remaining: 25.3s
10:	learn: 1.4855822	total: 269ms	remaining: 24.2s
11:	learn: 1.4746670	total: 285ms	remaining: 23.5s
12:	learn: 1.4630948	total: 301ms	remaining: 22.8s
13:	learn: 1.4541360	total: 316ms	remaining: 22.3s
14:	learn: 1.4463517	total: 328ms	remaining: 21.5s
15:	learn: 1.4379874	total: 340ms	remaining: 20.9s
16:	learn: 1.4312984	total: 352ms	remaining: 20.3s
17:	learn: 1.4244472	total: 364ms	remaining: 19.8s
18:	learn: 1.4182308	total: 376ms	remaining: 19.4s
19:	learn: 1

167:	learn: 1.2488163	total: 2.75s	remaining: 13.6s
168:	learn: 1.2481993	total: 2.77s	remaining: 13.6s
169:	learn: 1.2475393	total: 2.78s	remaining: 13.6s
170:	learn: 1.2471429	total: 2.8s	remaining: 13.6s
171:	learn: 1.2465937	total: 2.81s	remaining: 13.5s
172:	learn: 1.2458013	total: 2.83s	remaining: 13.5s
173:	learn: 1.2452573	total: 2.85s	remaining: 13.5s
174:	learn: 1.2444623	total: 2.88s	remaining: 13.6s
175:	learn: 1.2440848	total: 2.9s	remaining: 13.6s
176:	learn: 1.2437079	total: 2.92s	remaining: 13.6s
177:	learn: 1.2429372	total: 2.94s	remaining: 13.6s
178:	learn: 1.2423371	total: 2.98s	remaining: 13.7s
179:	learn: 1.2418332	total: 3s	remaining: 13.7s
180:	learn: 1.2410890	total: 3.02s	remaining: 13.7s
181:	learn: 1.2404317	total: 3.03s	remaining: 13.6s
182:	learn: 1.2397291	total: 3.04s	remaining: 13.6s
183:	learn: 1.2394656	total: 3.06s	remaining: 13.6s
184:	learn: 1.2388224	total: 3.07s	remaining: 13.5s
185:	learn: 1.2380259	total: 3.09s	remaining: 13.5s
186:	learn: 1.237

328:	learn: 1.1682023	total: 5.26s	remaining: 10.7s
329:	learn: 1.1679397	total: 5.28s	remaining: 10.7s
330:	learn: 1.1676133	total: 5.29s	remaining: 10.7s
331:	learn: 1.1670717	total: 5.3s	remaining: 10.7s
332:	learn: 1.1667165	total: 5.32s	remaining: 10.6s
333:	learn: 1.1664394	total: 5.33s	remaining: 10.6s
334:	learn: 1.1655540	total: 5.34s	remaining: 10.6s
335:	learn: 1.1651298	total: 5.35s	remaining: 10.6s
336:	learn: 1.1646512	total: 5.36s	remaining: 10.6s
337:	learn: 1.1642248	total: 5.38s	remaining: 10.5s
338:	learn: 1.1637779	total: 5.39s	remaining: 10.5s
339:	learn: 1.1633501	total: 5.4s	remaining: 10.5s
340:	learn: 1.1628173	total: 5.41s	remaining: 10.5s
341:	learn: 1.1625571	total: 5.42s	remaining: 10.4s
342:	learn: 1.1621270	total: 5.43s	remaining: 10.4s
343:	learn: 1.1616627	total: 5.45s	remaining: 10.4s
344:	learn: 1.1612651	total: 5.46s	remaining: 10.4s
345:	learn: 1.1609977	total: 5.48s	remaining: 10.4s
346:	learn: 1.1605838	total: 5.49s	remaining: 10.3s
347:	learn: 1.

488:	learn: 1.1086908	total: 7.31s	remaining: 7.64s
489:	learn: 1.1084231	total: 7.33s	remaining: 7.63s
490:	learn: 1.1078647	total: 7.35s	remaining: 7.62s
491:	learn: 1.1074809	total: 7.37s	remaining: 7.61s
492:	learn: 1.1072526	total: 7.38s	remaining: 7.59s
493:	learn: 1.1068333	total: 7.39s	remaining: 7.57s
494:	learn: 1.1065954	total: 7.4s	remaining: 7.55s
495:	learn: 1.1063603	total: 7.41s	remaining: 7.53s
496:	learn: 1.1060834	total: 7.42s	remaining: 7.51s
497:	learn: 1.1057824	total: 7.43s	remaining: 7.49s
498:	learn: 1.1056226	total: 7.45s	remaining: 7.48s
499:	learn: 1.1052931	total: 7.46s	remaining: 7.46s
500:	learn: 1.1048735	total: 7.47s	remaining: 7.44s
501:	learn: 1.1046148	total: 7.49s	remaining: 7.43s
502:	learn: 1.1042572	total: 7.5s	remaining: 7.41s
503:	learn: 1.1039118	total: 7.52s	remaining: 7.4s
504:	learn: 1.1035105	total: 7.54s	remaining: 7.39s
505:	learn: 1.1032833	total: 7.55s	remaining: 7.37s
506:	learn: 1.1029987	total: 7.56s	remaining: 7.35s
507:	learn: 1.1

647:	learn: 1.0592782	total: 9.38s	remaining: 5.09s
648:	learn: 1.0588194	total: 9.39s	remaining: 5.08s
649:	learn: 1.0584045	total: 9.41s	remaining: 5.07s
650:	learn: 1.0579980	total: 9.43s	remaining: 5.05s
651:	learn: 1.0576715	total: 9.44s	remaining: 5.04s
652:	learn: 1.0574443	total: 9.45s	remaining: 5.02s
653:	learn: 1.0571357	total: 9.46s	remaining: 5s
654:	learn: 1.0569155	total: 9.47s	remaining: 4.99s
655:	learn: 1.0564012	total: 9.48s	remaining: 4.97s
656:	learn: 1.0561780	total: 9.5s	remaining: 4.96s
657:	learn: 1.0558898	total: 9.51s	remaining: 4.94s
658:	learn: 1.0557546	total: 9.52s	remaining: 4.93s
659:	learn: 1.0554466	total: 9.53s	remaining: 4.91s
660:	learn: 1.0551954	total: 9.54s	remaining: 4.89s
661:	learn: 1.0549415	total: 9.55s	remaining: 4.88s
662:	learn: 1.0547606	total: 9.57s	remaining: 4.86s
663:	learn: 1.0544284	total: 9.58s	remaining: 4.85s
664:	learn: 1.0540808	total: 9.6s	remaining: 4.83s
665:	learn: 1.0538481	total: 9.61s	remaining: 4.82s
666:	learn: 1.053

...

285:	learn: 1.2076944	total: 5.39s	remaining: 13.5s
286:	learn: 1.2074495	total: 5.41s	remaining: 13.4s
287:	learn: 1.2069825	total: 5.43s	remaining: 13.4s
288:	learn: 1.2067256	total: 5.45s	remaining: 13.4s
289:	learn: 1.2063953	total: 5.46s	remaining: 13.4s
290:	learn: 1.2060250	total: 5.48s	remaining: 13.4s
291:	learn: 1.2057301	total: 5.49s	remaining: 13.3s
292:	learn: 1.2054704	total: 5.51s	remaining: 13.3s
293:	learn: 1.2050394	total: 5.53s	remaining: 13.3s
294:	learn: 1.2044766	total: 5.54s	remaining: 13.2s
295:	learn: 1.2041674	total: 5.56s	remaining: 13.2s
296:	learn: 1.2038241	total: 5.57s	remaining: 13.2s
297:	learn: 1.2033352	total: 5.59s	remaining: 13.2s
298:	learn: 1.2028584	total: 5.61s	remaining: 13.2s
299:	learn: 1.2024941	total: 5.63s	remaining: 13.1s
300:	learn: 1.2020731	total: 5.65s	remaining: 13.1s
301:	learn: 1.2016742	total: 5.66s	remaining: 13.1s
302:	learn: 1.2012133	total: 5.68s	remaining: 13.1s
303:	learn: 1.2007744	total: 5.69s	remaining: 13s
304:	learn: 1.

444:	learn: 1.1537768	total: 8.11s	remaining: 10.1s
445:	learn: 1.1535884	total: 8.15s	remaining: 10.1s
446:	learn: 1.1533570	total: 8.18s	remaining: 10.1s
447:	learn: 1.1530094	total: 8.2s	remaining: 10.1s
448:	learn: 1.1527200	total: 8.22s	remaining: 10.1s
449:	learn: 1.1523718	total: 8.25s	remaining: 10.1s
450:	learn: 1.1521532	total: 8.27s	remaining: 10.1s
451:	learn: 1.1519601	total: 8.29s	remaining: 10.1s
452:	learn: 1.1516010	total: 8.32s	remaining: 10s
453:	learn: 1.1514382	total: 8.35s	remaining: 10s
454:	learn: 1.1511663	total: 8.38s	remaining: 10s
455:	learn: 1.1507538	total: 8.4s	remaining: 10s
456:	learn: 1.1504791	total: 8.43s	remaining: 10s
457:	learn: 1.1500911	total: 8.45s	remaining: 10s
458:	learn: 1.1497990	total: 8.47s	remaining: 9.98s
459:	learn: 1.1494134	total: 8.49s	remaining: 9.97s
460:	learn: 1.1491826	total: 8.51s	remaining: 9.95s
461:	learn: 1.1487034	total: 8.54s	remaining: 9.95s
462:	learn: 1.1483890	total: 8.56s	remaining: 9.93s
463:	learn: 1.1481465	tota

605:	learn: 1.1097015	total: 12.5s	remaining: 8.1s
606:	learn: 1.1094777	total: 12.5s	remaining: 8.08s
607:	learn: 1.1093306	total: 12.5s	remaining: 8.07s
608:	learn: 1.1090182	total: 12.5s	remaining: 8.04s
609:	learn: 1.1086877	total: 12.5s	remaining: 8.02s
610:	learn: 1.1083444	total: 12.6s	remaining: 8s
611:	learn: 1.1079992	total: 12.6s	remaining: 7.97s
612:	learn: 1.1077859	total: 12.6s	remaining: 7.95s
613:	learn: 1.1075171	total: 12.6s	remaining: 7.92s
614:	learn: 1.1073879	total: 12.6s	remaining: 7.9s
615:	learn: 1.1070698	total: 12.6s	remaining: 7.88s
616:	learn: 1.1067342	total: 12.7s	remaining: 7.85s
617:	learn: 1.1065414	total: 12.7s	remaining: 7.83s
618:	learn: 1.1062602	total: 12.7s	remaining: 7.81s
619:	learn: 1.1060382	total: 12.7s	remaining: 7.79s
620:	learn: 1.1058131	total: 12.7s	remaining: 7.76s
621:	learn: 1.1056372	total: 12.7s	remaining: 7.74s
622:	learn: 1.1054268	total: 12.8s	remaining: 7.71s
623:	learn: 1.1051775	total: 12.8s	remaining: 7.69s
624:	learn: 1.104

765:	learn: 1.0717583	total: 17.4s	remaining: 5.32s
766:	learn: 1.0715610	total: 17.4s	remaining: 5.3s
767:	learn: 1.0713974	total: 17.5s	remaining: 5.28s
768:	learn: 1.0712371	total: 17.5s	remaining: 5.25s
769:	learn: 1.0710376	total: 17.5s	remaining: 5.22s
770:	learn: 1.0709227	total: 17.5s	remaining: 5.2s
771:	learn: 1.0706855	total: 17.5s	remaining: 5.18s
772:	learn: 1.0705815	total: 17.5s	remaining: 5.15s
773:	learn: 1.0703559	total: 17.6s	remaining: 5.13s
774:	learn: 1.0701817	total: 17.6s	remaining: 5.1s
775:	learn: 1.0699139	total: 17.6s	remaining: 5.08s
776:	learn: 1.0697110	total: 17.6s	remaining: 5.05s
777:	learn: 1.0695869	total: 17.6s	remaining: 5.03s
778:	learn: 1.0694435	total: 17.7s	remaining: 5.01s
779:	learn: 1.0692979	total: 17.7s	remaining: 4.98s
780:	learn: 1.0691105	total: 17.7s	remaining: 4.96s
781:	learn: 1.0689340	total: 17.7s	remaining: 4.93s
782:	learn: 1.0687103	total: 17.7s	remaining: 4.91s
783:	learn: 1.0684289	total: 17.7s	remaining: 4.89s
784:	learn: 1.0

928:	learn: 1.0388675	total: 21.4s	remaining: 1.64s
929:	learn: 1.0387196	total: 21.5s	remaining: 1.62s
930:	learn: 1.0384136	total: 21.5s	remaining: 1.59s
931:	learn: 1.0382450	total: 21.5s	remaining: 1.57s
932:	learn: 1.0380885	total: 21.6s	remaining: 1.55s
933:	learn: 1.0378156	total: 21.6s	remaining: 1.53s
934:	learn: 1.0374588	total: 21.6s	remaining: 1.5s
935:	learn: 1.0372742	total: 21.7s	remaining: 1.48s
936:	learn: 1.0371013	total: 21.7s	remaining: 1.46s
937:	learn: 1.0369436	total: 21.7s	remaining: 1.44s
938:	learn: 1.0368393	total: 21.8s	remaining: 1.41s
939:	learn: 1.0367102	total: 21.8s	remaining: 1.39s
940:	learn: 1.0365132	total: 21.8s	remaining: 1.37s
941:	learn: 1.0364122	total: 21.8s	remaining: 1.34s
942:	learn: 1.0361892	total: 21.9s	remaining: 1.32s
943:	learn: 1.0360136	total: 21.9s	remaining: 1.3s
944:	learn: 1.0358780	total: 21.9s	remaining: 1.28s
945:	learn: 1.0356300	total: 22s	remaining: 1.25s
946:	learn: 1.0354944	total: 22s	remaining: 1.23s
947:	learn: 1.0353

#### SVM

In [194]:
svc = SVC()

predictions(svc)

Train Metrics

[[ 818 1356   69    0    0   30]
 [ 475 2582   88    0    0   37]
 [ 480 2276   88    0    0   27]
 [ 193  835   33    0    0   11]
 [ 495 1420   43    0    0   41]
 [ 377 1980   62    0    0   44]] 


Accuracy Score: 0.25483405483405486 


              precision    recall  f1-score   support

           0       0.29      0.36      0.32      2273
           1       0.25      0.81      0.38      3182
           2       0.23      0.03      0.05      2871
           3       0.00      0.00      0.00      1072
           4       0.00      0.00      0.00      1999
           5       0.23      0.02      0.03      2463

    accuracy                           0.25     13860
   macro avg       0.17      0.20      0.13     13860
weighted avg       0.19      0.25      0.16     13860
 


Test Metrics

[[ 353  583   23    0    0   13]
 [ 217 1052   33    0    0   16]
 [ 216  991   34    0    0   13]
 [ 110  379   11    0    0    3]
 [ 203  586   18    0    0   18]
 [ 147  878   23   